<a href="https://colab.research.google.com/github/chaitraDev/GenAI-for-marketing/blob/main/Cognizant_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
num_clusters = 51

# Setup

In [ ]:
import pandas as pd

In [ ]:
customers = pd.read_csv("/content/drive/MyDrive/Cognizant/customers_segmented_p2.csv")

In [ ]:
customers.columns

Index(['Unnamed: 0', 'CustomerID', 'CustGender', 'CustLocation',
       'CustAccountBalance', 'most_recent', 'Gap', 'Num_Transactions',
       'Sum_Transactions', 'Recency', 'Monetary_Value', 'Frequency',
       'RFM_Score', 'Relationship_Value', 'Age', 'Cibil_Score', 'Income',
       'Product'],
      dtype='object')

In [ ]:
# customers = customers.iloc[:rows,:]

In [ ]:
# the columns we need gender, age, income, cibil, relationship value
customers_req = customers[[ 'Age', 'Income','Cibil_Score', 'Relationship_Value', 'Product']]

In [ ]:
customers[customers.Relationship_Value.isnull()]

,Unnamed: 0,CustomerID,CustGender,CustLocation,CustAccountBalance,most_recent,Gap,Num_Transactions,Sum_Transactions,Recency,Monetary_Value,Frequency,RFM_Score,Relationship_Value,Age,Cibil_Score,Income,Product


# Transforming

In [ ]:
customers_req.select_dtypes('O').columns

Index(['Relationship_Value', 'Product'], dtype='object')

- Relationship_Value is ordinal
- Gender, Location is nominal

## Gender Column

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# OHE = OneHotEncoder()
# encoded = OHE.fit_transform(customers_req[["CustGender"]]).toarray()
# customers_req["Gender(num)"] = OHE.fit_transform(customers_req[["CustGender"]])

In [ ]:
# print(encoded[0])
# customers_req["CustGender"].iloc[0]
#encoded as (female,male)

In [ ]:
# customers_req.loc[:,"IsFemale"] = encoded[:,0]
# customers_req[["CustGender","IsFemale"]].head(2)

## Relationship value column

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# lbl_encoder = LabelEncoder()
# customers_req.loc[:,"Relationship_Value(num)"] = lbl_encoder.fit_transform(customers_req["Relationship_Value"])
customers_req.loc[:,"Relationship_Value(num)"] = customers_req["Relationship_Value"].map({"high":2,"med":1,"low":0})

<ipython-input-13-f1de21af0a84>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_req.loc[:,"Relationship_Value(num)"] = customers_req["Relationship_Value"].map({"high":2,"med":1,"low":0})


In [ ]:
customers_req[["Relationship_Value","Relationship_Value(num)"]].head(4)

,Relationship_Value,Relationship_Value(num)
0,high,2
1,low,0
2,low,0
3,low,0


In [ ]:
customers_req.columns

Index(['Age', 'Income', 'Cibil_Score', 'Relationship_Value', 'Product',
       'Relationship_Value(num)'],
      dtype='object')

## Products Column

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbl_encoder = LabelEncoder()
customers_req.loc[:,"Product"] = lbl_encoder.fit_transform(customers_req["Product"])


# Normalization

In [ ]:
cust_numeric = customers_req[["Age","Income","Cibil_Score","Relationship_Value(num)","Product"]]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cust_numeric = pd.DataFrame(scaler.fit_transform(cust_numeric),columns=cust_numeric.columns)

In [ ]:
cust_numeric.isnull().sum()

,0
Age,0
Income,0
Cibil_Score,0
Relationship_Value(num),0
Product,0


# K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans
X = cust_numeric

In [ ]:
def score_kmeans(k):
  kmeans = KMeans(k,random_state=42)
  kmeans.fit(X)
  return silhouette_score(X,kmeans.labels_)

In [ ]:
kmeans = KMeans(num_clusters,random_state=144)
kmeans.fit(X)
clusters = kmeans.labels_
cust_numeric["clusters"] = clusters

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# Scoring

In [ ]:
from sklearn.metrics import davies_bouldin_score as dbs

In [ ]:
print(dbs(X,kmeans.labels_))

0.5490804345810382


# Saving

In [ ]:
cust_numeric.to_csv("cust_segmented_clustered.csv")

In [ ]:
import joblib
joblib.dump(kmeans,"/content/kmeans1.pkl")

['/content/kmeans1.pkl']